# GD-3. Исследование поведения пользователей 

1. Рассмотрим, какие бывают паттерны поведения у пользователей.
2. Выясним, как эти паттерны влияют на конверсию в оплату и сумму покупок.
3. Продолжим развивать навыки работы с Python для решения аналитических задач.
4. Самостоятельно проверим две гипотезы и оценим работу коллег.

## 38.1 THEORY. Анализ функциональности

Фича (от англ. feature, «особенность») — некая «полезность» продукта, которая отличает его от других продуктов этой же категории.

Функциональность — любая возможность или функция, которую предоставляет продукт.

## 38.2 SETTING. Ваши задачи

#### ЦЕЛЬ
Исследовать поведение пользователей в обновлённом приложении.



#### ЗАДАЧИ
1. Понять, как пользователи взаимодействуют с продуктом, и соотнести идеальный путь пользователей с фактическим.
2. Выявить этапы, которые занимают больше всего времени, с тем чтобы поработать над их улучшением.
3. Проанализировать зависимость оплат от прохождения обучения.

#### КОНКРЕТНЫЕ ШАГИ (ФОРМАЛИЗОВАННЫЕ ЗАДАЧИ)
1. Определить самые распространённые пути прохождения (последовательности) этапов в приложении.
2. Посмотреть на среднее время между различными этапами и выделить самые большие временные промежутки.
3. Определить, существует ли различие в частоте и средней величине оплат между тремя группами пользователей:

    a) пользователями, которые прошли обучение хотя бы раз;

    b) пользователями, которые начали обучение, но не прошли его ни разу;

    c) пользователями, которые не начинали обучение, а сразу же перешли к выбору уровня сложности.

## 38.3 SETTING. Знакомство с данными

In [21]:
import pandas as pd

### Таблица Event
Хранит данные о событиях, которые совершают пользователи. По сути, каждое событие — это факт прохождения пользователем какого-либо этапа игры.

id	идентификатор события

user_id	уникальный идентификатор пользователя, совершившего событие в приложении

start_time	дата и время события

event_type	тип события (значения: registration — регистрация; tutorial_start — начало обучения; tutorial_finish — завершение обучения; level_choice — выбор уровня сложности; pack_choice — выбор пакетов вопросов)

tutorial_id	идентификатор обучения (этот идентификатор есть только у событий обучения)

selected_level	выбранный уровень сложности обучения


In [44]:
event = pd.read_csv('data/7_4_Events.csv')
event.head()

,id,event_type,selected_level,start_time,tutorial_id,user_id
0,28903,registration,NaN,2016-05-11T23:40:55,NaN,12583
1,28904,registration,NaN,2016-05-11T23:49:58,NaN,12584
2,28905,registration,NaN,2016-05-12T00:53:07,NaN,12585
3,28906,tutorial_start,NaN,2016-05-12T01:32:20,17562.0,12585
4,28907,tutorial_finish,NaN,2016-05-12T01:34:53,17562.0,12585


In [45]:
# условие регестрации в 2018
mask = (event['start_time']>='2018-01-01') & (event['start_time']<'2019-01-01') & (event['event_type']=='registration')
# список пользователей, зарег. в 2018
registered = event[mask]['user_id'].to_list()
# новая таблица events. Метод isin фильтрация по списку значений
events_df = event[event['user_id'].isin(registered)]
# приведение даты к формату
events_df['start_time'] = pd.to_datetime(events_df['start_time'])
#просмотр результата
events_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 66959 entries, 51405 to 118364
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              66959 non-null  int64         
 1   event_type      66959 non-null  object        
 2   selected_level  8342 non-null   object        
 3   start_time      66959 non-null  datetime64[ns]
 4   tutorial_id     32954 non-null  float64       
 5   user_id         66959 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 3.6+ MB


<ipython-input-45-3c377d7b8a26>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_df['start_time'] = pd.to_datetime(events_df['start_time'])


#### Исследуем пропуски

In [46]:
# посмотрим на events_df, оставив в нём только такие строки, где event_type = level_choice.
events_df[events_df['event_type'] == 'level_choice'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8342 entries, 51424 to 118363
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              8342 non-null   int64         
 1   event_type      8342 non-null   object        
 2   selected_level  8342 non-null   object        
 3   start_time      8342 non-null   datetime64[ns]
 4   tutorial_id     0 non-null      float64       
 5   user_id         8342 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 456.2+ KB


In [47]:
# Теперь проверим аналогичные данные, но при условии, что срез будет содержать данные
# о событиях tutorial_start и tutorial_finish.
events_df[events_df['event_type'].isin(['tutorial_start','tutorial_finish'])].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32954 entries, 51412 to 118360
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              32954 non-null  int64         
 1   event_type      32954 non-null  object        
 2   selected_level  0 non-null      object        
 3   start_time      32954 non-null  datetime64[ns]
 4   tutorial_id     32954 non-null  float64       
 5   user_id         32954 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 1.8+ MB


#### Исследуем значения

In [48]:
# описательная статистика
"""Вызовем метод describe(), чтобы оценить характеристики каждого столбца. 
По умолчанию метод describe() выдаёт характеристики только по столбцам с численными 
типами (например, int64, float64). Мы вызовем этот метод с параметром include='all'
для того, чтобы отображать характеристики для всех столбцов."""

events_df.describe(include='all')

<ipython-input-48-e8ecb979889f>:7: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  events_df.describe(include='all')


,id,event_type,selected_level,start_time,tutorial_id,user_id
count,66959.000000,66959,8342,66959,32954.000000,66959.000000
unique,NaN,5,3,66809,NaN,NaN
top,NaN,registration,medium,2018-03-13 09:37:43,NaN,NaN
freq,NaN,19926,4645,9,NaN,NaN
first,NaN,NaN,NaN,2018-01-01 03:48:40,NaN,NaN
last,NaN,NaN,NaN,2019-01-01 05:50:36,NaN,NaN
mean,113787.000045,NaN,NaN,NaN,40532.934393,37781.543362
std,19329.542752,NaN,NaN,NaN,5213.486632,5751.497904
min,80308.000000,NaN,NaN,NaN,31505.000000,27832.000000
25%,97047.500000,NaN,NaN,NaN,36008.250000,32849.000000


In [49]:
# Давайте оценим, какие уникальные события есть в колонках event_type и selected_level.
display(events_df['event_type'].unique())
display(events_df['selected_level'].unique())

array(['registration', 'tutorial_start', 'tutorial_finish',
       'level_choice', 'pack_choice'], dtype=object)

array([nan, 'medium', 'hard', 'easy'], dtype=object)

In [50]:
# Также оценим, какое количество пользователей совершали события:
events_df['user_id'].nunique()

19926

### Таблица purchase
Хранит данные об оплатах, которые совершают пользователи.

id	идентификатор события

user_id	уникальный идентификатор пользователя, совершившего событие в приложении

event_datetime	дата и время события/покупки

amount	сумма оплаты

In [51]:
purchase = pd.read_csv('data/purchase.csv')
display(purchase.head())
display(purchase.info())

,id,user_id,event_datetime,amount
0,15674,12584,2016-05-12T10:34:16,100
1,15675,12985,2016-05-13T08:25:56,50
2,15676,12828,2016-05-13T16:33:46,50
3,15677,12598,2016-05-14T01:09:37,150
4,15678,13037,2016-05-14T01:24:46,100


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5956 entries, 0 to 5955
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              5956 non-null   int64 
 1   user_id         5956 non-null   int64 
 2   event_datetime  5956 non-null   object
 3   amount          5956 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 186.2+ KB


None

In [116]:
# новая таблица purchase. Метод isin фильтрация по списку значений (список user_id определен выше)
purchase_df = purchase[purchase['user_id'].isin(registered)]
# приведение даты к формату
purchase_df['event_datetime'] = pd.to_datetime(purchase_df['event_datetime'])
#просмотр результата
purchase_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600 entries, 1171 to 2778
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              1600 non-null   int64         
 1   user_id         1600 non-null   int64         
 2   event_datetime  1600 non-null   datetime64[ns]
 3   amount          1600 non-null   int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 62.5 KB


<ipython-input-116-610ab4965a64>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchase_df['event_datetime'] = pd.to_datetime(purchase_df['event_datetime'])


#### Исследуем значения

In [117]:
purchase_df.describe()

,id,user_id,amount
count,1600.000000,1600.000000,1600.000000
mean,17645.505625,37752.766250,110.734375
std,462.038637,5822.621784,54.696628
min,16845.000000,27845.000000,25.000000
25%,17245.750000,32815.750000,50.000000
50%,17645.500000,37633.500000,100.000000
75%,18045.250000,43023.000000,150.000000
max,18452.000000,47742.000000,300.000000


## 38.5 PRACTICE. Анализ данных: Шаг 1

### АНАЛИЗ ПОЛЬЗОВАТЕЛЬСКИХ СОБЫТИЙ

цель — исследовать поведение пользователей:

1. Определить самые распространённые пути прохождения этапов в приложении.
2. Определить среднее время между различными этапами.

#### Изучаем событие registration

In [55]:
# Для начала посмотрим, сколько пользователей совершают событие registration.
print(events_df[events_df['event_type'] == 'registration']['user_id'].nunique())
print(events_df['user_id'].nunique())

19926
19926


#### Изучаем событие tutorial_start

In [56]:
# Посмотрим на срез данных по событию tutorial_start.
events_df[events_df['event_type'] == 'tutorial_start'].head(10)

,id,event_type,selected_level,start_time,tutorial_id,user_id
51412,80315,tutorial_start,NaN,2018-01-01 14:54:40,31505.0,27836
51413,80316,tutorial_start,NaN,2018-01-01 15:00:51,31506.0,27835
51415,80318,tutorial_start,NaN,2018-01-01 15:40:43,31507.0,27836
51417,80320,tutorial_start,NaN,2018-01-01 17:47:40,31508.0,27833
51420,80323,tutorial_start,NaN,2018-01-01 19:11:36,31509.0,27839
51422,80325,tutorial_start,NaN,2018-01-01 19:46:11,31510.0,27834
51434,80337,tutorial_start,NaN,2018-01-02 02:07:07,31511.0,27840
51435,80338,tutorial_start,NaN,2018-01-02 03:03:44,31512.0,27845
51437,80340,tutorial_start,NaN,2018-01-02 04:55:11,31513.0,27842
51446,80349,tutorial_start,NaN,2018-01-02 07:08:00,31514.0,27845


In [58]:
# Посмотрим на количество пользователей, которые совершают событие tutorial_start:
events_df[events_df['event_type'] == 'tutorial_start']['user_id'].nunique()

11858

In [59]:
# определим процент пользователей, которые перешли к выполнению обучения, и 
# запишем его в переменную percent_tutorial_start_users.

registered_users_count = events_df[events_df["event_type"] == "registration"][
    "user_id"
].nunique()
tutorial_start_users_count = events_df[events_df["event_type"] == "tutorial_start"][
    "user_id"
].nunique()
percent_tutorial_start_users = tutorial_start_users_count / registered_users_count
print(
    "Процент пользователей, начавших обучение (от общего числа зарегистрировавшихся): {:.2%}".format(
        percent_tutorial_start_users
    )
)

Процент пользователей, начавших обучение (от общего числа зарегистрировавшихся): 59.51%


#### Изучаем событие tutorial_finish

In [60]:
# Теперь давайте посмотрим, какое количество пользователей проходит обучение 
# до конца (событие tutorial_finish).
events_df[events_df['event_type'] == 'tutorial_finish']['user_id'].nunique()

10250

In [61]:
# Рассчитаем процент пользователей, завершивших обучение, среди пользователей, 
# которые начали обучение. Это будет показатель tutorial_completion_rate 
# (коэффициент «завершаемости» обучения).

tutorial_finish_users_count = events_df[events_df["event_type"] == "tutorial_finish"][
    "user_id"
].nunique()
tutorial_completion_rate = tutorial_finish_users_count / tutorial_start_users_count
print(
    "Процент пользователей, завершивших обучение: {:.2%}".format(
        tutorial_completion_rate
    )
)

Процент пользователей, завершивших обучение: 86.44%


#### Изучаем level_choice

In [62]:
# посмотрим на процент тех, кто доходит до этого этапа.
events_df[events_df['event_type'] == 'level_choice']['user_id'].nunique()

8342

In [63]:
# рассчитаем долю пользователей percent_level_choice_users, которые выбрали уровень 
# сложности, от общего числа зарегистрировавшихся:

level_choice_users_count = events_df[events_df["event_type"] == "level_choice"][
    "user_id"
].nunique()
percent_level_choice_users = level_choice_users_count / registered_users_count
print(
    "Процент пользователей, выбравших уровень сложности тренировок (от общего числа зарегистрировавшихся): {:.2%}".format(
        percent_level_choice_users
    )
)

Процент пользователей, выбравших уровень сложности тренировок (от общего числа зарегистрировавшихся): 41.86%


Какой вывод мы можем сделать?
Меньше половины пользователей (41.86 %) доходят до этапа выбора уровня сложности вопросов. А ведь этот этап напрямую влияет на то, что пользователь будет пользоваться приложением через бесплатные возможности, которые в дальнейшем могут привести к оплате.

Таким образом, для успешной монетизации приложения крайне важно оптимизировать прохождение до этапа выбора сложности.

####  Изучаем pack_choice

In [64]:
# определим число пользователей, которые совершили событие выбора бесплатного 
# пакета вопросов pack_choice.
events_df[events_df['event_type'] == 'pack_choice']['user_id'].nunique()

5737

In [65]:
# Оценим процент таких пользователей от числа пользователей, которые выбрали 
# уровень сложности.

training_choice_users_count = events_df[events_df["event_type"] == "pack_choice"][
    "user_id"
].nunique()
percent_training_choice_users = training_choice_users_count / level_choice_users_count
print(
    "Процент пользователей, выбравших набор бесплатных вопросов (от числа пользователей, которые выбрали уровень сложности): {:.2%}".format(
        percent_training_choice_users
    )
)

Процент пользователей, выбравших набор бесплатных вопросов (от числа пользователей, которые выбрали уровень сложности): 68.77%


#### Изучаем покупку платных пакетов

In [66]:
#Оценим число пользователей, которые совершили покупку. Для этого нам потребуется 
# датафрейм purchase_df.
purchase_df['user_id'].nunique()

1600

In [67]:
# Рассчитаем процент пользователей percent_of_paying_users, которые оплатили 
# вопросы, от числа пользователей, которые приобрели бесплатные вопросы:

paying_users_count = purchase_df["user_id"].nunique()
percent_of_paying_users = paying_users_count / training_choice_users_count
print(
    "Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали тренировки): {:.2%}".format(
        percent_of_paying_users
    )
)

Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали тренировки): 27.89%


In [68]:
# посмотрим, какой процент составляют покупатели от общего числа зарегистрировавшихся. 
# Для этого создадим переменную purchase_rate.

purchase_rate = paying_users_count / registered_users_count
print(
    "Процент пользователей, которые оплатили вопросы(от числа зарегистрировавшихся пользователей): {:.2%}".format(
        purchase_rate
    )
)

Процент пользователей, которые оплатили вопросы(от числа зарегистрировавшихся пользователей): 8.03%


 Какой вывод мы можем сделать?
Итак, процент зарегистрировавшихся пользователей, которые приобрели платный пакет, составляет 8.03 %.

Само по себе это число нам ни о чём не говорит, поэтому в рамках одной из гипотез мы рассмотрим этот процент для пользователей разных групп. И эта информация укажет нам возможные пути оптимизации.

## 38.6 PRACTICE. Анализ данных: Шаг 2

### АНАЛИЗ УНИКАЛЬНЫХ ПОЛЬЗОВАТЕЛЬСКИХ ПУТЕЙ

Чтобы понимать, какие есть различные последовательности прохождения пользователей по воронке и насколько часто они встречаются, можно воспользоваться таким подходом:

1. отсортировать все события по возрастанию во времени;
2. объединить для каждого пользователя все его события в один список;
3. подсчитать частоту различных списков.

#### Объединяем датафреймы

In [ ]:
"""Добавим в датафрейм purchase_df столбец event_type, который будет содержать одно 
значение purchase. Это нужно, чтобы в объединённом датафрейме однозначно выделить 
события оплаты."""

purchase_df['event_type'] = 'purchase'

In [ ]:
purchase_df.drop('event_type', axis=1, inplace=True)

In [81]:
"""Также у нас есть одинаковые столбцы id в двух датафреймах, но смысл их 
несколько отличается, так как столбец id в events_df указывает на идентификатор 
события, а столбец id в purchase_df указывает на идентификатор оплаты. Поэтому 
применим функцию rename(), чтобы переименовать столбцы в датафреймах."""

events_df = events_df.rename(columns={"id": "event_id"})
purchase_df = purchase_df.rename(columns={"id": "purchase_id"})

In [83]:
"""объединим датафреймы events_df и purchase_df с помощью функции pd.concat() 
и запишем объединённый датафрейм в переменную total_events_df."""

total_events_df = pd.concat([events_df,purchase_df],sort=False)

In [ ]:
total_events_df.head()

In [85]:
total_events_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68559 entries, 51405 to 2778
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_id        66959 non-null  float64       
 1   event_type      68559 non-null  object        
 2   selected_level  8342 non-null   object        
 3   start_time      66959 non-null  datetime64[ns]
 4   tutorial_id     32954 non-null  float64       
 5   user_id         68559 non-null  int64         
 6   purchase_id     1600 non-null   float64       
 7   event_datetime  1600 non-null   datetime64[ns]
 8   amount          1600 non-null   float64       
dtypes: datetime64[ns](2), float64(4), int64(1), object(2)
memory usage: 5.2+ MB


In [86]:
"""сбросим индексы объединённого датафрейма (так как после объединения они 
дублировались и несут мало смысла) с помощью метода reset_index() и отсортируем 
все события по возрастанию времени с помощью sort_values()."""

total_events_df = total_events_df.reset_index(drop=True).sort_values('start_time')

####  Воссоздаём последовательность событий

In [87]:
"""воспользуемся методом groupby по столбцу event_type и применим агрегирующую 
функцию apply(list). Таким образом мы сгруппируем строки по пользователю, а затем 
объединим в списки содержимое столбца event_type по каждому пользователю. Запишем 
результат в датафрейм user_path_df."""

user_path_df = (
    total_events_df.groupby(["user_id"])["event_type"].apply(list).reset_index()
)
user_path_df.head(10)

,user_id,event_type
0,27832,[registration]
1,27833,"[registration, tutorial_start, tutorial_finish]"
2,27834,"[registration, tutorial_start, tutorial_finish]"
3,27835,"[registration, tutorial_start, tutorial_finish..."
4,27836,"[registration, tutorial_start, tutorial_start,..."
5,27837,[registration]
6,27838,[registration]
7,27839,"[registration, tutorial_start, tutorial_finish..."
8,27840,"[registration, tutorial_start, level_choice]"
9,27841,"[registration, tutorial_start, tutorial_finish]"


#### Выявляем наиболее популярные пути

In [88]:
"""преобразуем список событий в строку event_path. Эта операция нужна для 
оптимизации скорости объединения, так как иначе Pandas может делать подсчёт 
слишком долго."""

user_path_df["event_path"] = user_path_df["event_type"].apply(lambda x: " > ".join(x))
user_path_df["event_path"].head()

0                                         registration
1      registration > tutorial_start > tutorial_finish
2      registration > tutorial_start > tutorial_finish
3    registration > tutorial_start > tutorial_finis...
4    registration > tutorial_start > tutorial_start...
Name: event_path, dtype: object

In [89]:
"""сгруппировать датафрейм по столбцу event_path, подсчитав число пользователей."""

user_paths = (
    user_path_df.groupby(["event_path"])["user_id"]
    .nunique()
    .sort_values(ascending=False)
)

In [90]:
user_paths.head(10)

event_path
registration                                                                                                       7970
registration > tutorial_start > tutorial_finish > level_choice > pack_choice                                       2796
registration > tutorial_start > tutorial_finish                                                                    1956
registration > tutorial_start > tutorial_finish > level_choice                                                     1713
registration > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase                            1083
registration > tutorial_start                                                                                       842
registration > tutorial_start > level_choice > pack_choice                                                          346
registration > tutorial_start > tutorial_finish > tutorial_start > tutorial_finish                                  323
registration > tutorial_start

#### ВЫВОД
Как мы видим, среди 10 самых популярных последовательностей только одна содержит этап оплаты. Это последовательность registration > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase.

#### Ищем последовательности с оплатой

In [92]:
"""Давайте посмотрим, какие ещё последовательности содержат в себе оплату."""
user_paths[user_paths.index.str.contains('purchase')].head(5)

event_path
registration > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase                                       1083
registration > tutorial_start > level_choice > pack_choice > purchase                                                          124
registration > tutorial_start > tutorial_finish > level_choice > pack_choice > tutorial_start > tutorial_finish > purchase     101
registration > tutorial_start > tutorial_finish > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase      52
registration > tutorial_start > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase                        26
Name: user_id, dtype: int64

#### ВЫВОД
Большинство последовательностей, которые содержат в себе оплату, также содержат старт обучения. Это наводит нас на гипотезу, что вероятность оплаты зависит от того, проходил ли пользователь обучение.

## 38.7 PRACTICE. Анализ данных: Шаг 3

### АНАЛИЗ ВРЕМЕННЫХ ПРОМЕЖУТКОВ

#### Определяем время между registration и tutorial_start

Jпределим, какое время проходит между регистрацией и началом обучения.
Важно понимать, что это время мы можем определить только для таких пользователей, которые хотя бы раз начинали обучение.

1. registration

In [93]:
"""Первым делом выделим отдельный датафрейм registration_df, который будет содержать только 
события с event_type = registration. Этот датафрейм будет вспомогательным для определения 
времени между регистрацией и началом обучения."""

registration_df = total_events_df[total_events_df['event_type'] == 'registration']

In [94]:
"""Для определения среднего количества событий на пользователя мы можем воспользоваться 
методом value_counts(), результатом которого будет серия с количеством событий на каждого 
пользователя. Затем, усреднив эти значения между собой, мы получим среднее число событий 
типа registration на одного пользователя."""

registration_df['user_id'].value_counts().mean()

1.0

In [95]:
"""Оставим в датафрейме registration_df только те данные, которые нужны для наших вычислений — 
столбец user_id с идентификатором пользователя и столбец start_time со временем регистрации. 
Также переименуем столбец start_time в столбец registration_time для понятности."""

registration_df = registration_df[["user_id", "start_time"]].rename(
    columns={"start_time": "registration_time"}
)

2. tutorial_start

In [96]:
"""Теперь возьмёмся за следующее событие. Выделим отдельный датафрейм tutorial_start_df, 
который будет содержать только события с event_type = tutorial_start (начало обучения)."""

tutorial_start_df = total_events_df[total_events_df['event_type'] == 'tutorial_start']

In [97]:
"""Так же, как и в случае с регистрациями, посчитаем, сколько таких событий приходится на 
пользователя."""

tutorial_start_df['user_id'].value_counts().mean()

1.522179119581717

In [98]:
"""Создадим датафрейм tutorial_start_df_wo_duplicates, где будет присутствовать только 
первое обучение. Для этого отсортируем датафрейм по start_time, чтобы сначала шли более 
ранние события начала обучения, а затем удалим дубликаты по столбцу user_id. Таким образом, 
для каждого user_id останется только первое событие типа tutorial_start."""

tutorial_start_df_wo_duplicates = tutorial_start_df.sort_values(
    "start_time"
).drop_duplicates("user_id")

In [99]:
"""Проверим, что для каждого пользователя осталось только одно событие."""

tutorial_start_df_wo_duplicates['user_id'].value_counts().mean()

1.0

In [100]:
"""Так же, как и в случае с датафреймом registration_df, оставим только такие столбцы, 
которые пригодятся нам в дальнейшем. Это столбцы user_id, start_time, tutorial_id. 
Также переименуем колонку start_time в tutorial_start_time:"""

tutorial_start_df_wo_duplicates = tutorial_start_df_wo_duplicates[
    ["user_id", "tutorial_id", "start_time"]
].rename(columns={"start_time": "tutorial_start_time"})

####   Объединяем датафреймы

In [101]:
"""Объединить датафреймы мы можем с помощью функции merge(), объединяя данные по параметру 
user_id. Также нам нужно объединить данные с использованием параметра how='inner', это 
позволит в объединённом датафрейме оставить только такие идентификаторы пользователей, 
которые есть в обоих датафреймах. Вспомните SQL, это очень похоже на INNER JOIN."""

merged_df = registration_df.merge(
    tutorial_start_df_wo_duplicates, on="user_id", how="inner"
)
merged_df.head()

,user_id,registration_time,tutorial_id,tutorial_start_time
0,27833,2018-01-01 04:07:25,31508.0,2018-01-01 17:47:40
1,27834,2018-01-01 08:35:10,31510.0,2018-01-01 19:46:11
2,27835,2018-01-01 11:54:47,31506.0,2018-01-01 15:00:51
3,27836,2018-01-01 13:28:07,31505.0,2018-01-01 14:54:40
4,27839,2018-01-01 18:24:01,31509.0,2018-01-01 19:11:36


In [102]:
"""Сделаем столбец timedelta, в котором посчитаем разницу между временем начала обучения 
(tutorial_start_time) и временем регистрации (registration_time):"""

merged_df["timedelta"] = (
    merged_df["tutorial_start_time"] - merged_df["registration_time"]
)
merged_df.head()

,user_id,registration_time,tutorial_id,tutorial_start_time,timedelta
0,27833,2018-01-01 04:07:25,31508.0,2018-01-01 17:47:40,0 days 13:40:15
1,27834,2018-01-01 08:35:10,31510.0,2018-01-01 19:46:11,0 days 11:11:01
2,27835,2018-01-01 11:54:47,31506.0,2018-01-01 15:00:51,0 days 03:06:04
3,27836,2018-01-01 13:28:07,31505.0,2018-01-01 14:54:40,0 days 01:26:33
4,27839,2018-01-01 18:24:01,31509.0,2018-01-01 19:11:36,0 days 00:47:35


In [103]:
"""Усреднив с помощью функции mean() значения в столбце timedelta, мы получим среднее время,
которое проходит между регистрацией пользователя и началом первого обучения:"""

merged_df["timedelta"].mean()

Timedelta('0 days 04:38:24.019817844')

In [104]:
"""Давайте воспользуемся методом describe(), чтобы понять некоторые важные параметры."""

merged_df['timedelta'].describe()

count                        11858
mean     0 days 04:38:24.019817844
std      0 days 04:15:09.650705034
min                0 days 00:00:34
25%         0 days 01:21:37.500000
50%         0 days 03:22:08.500000
75%         0 days 06:47:42.750000
max                1 days 16:03:46
Name: timedelta, dtype: object

#### Таким образом:

четверть пользователей тратит меньше 1 часа 26 минут на переход от регистрации к началу обучения;
половина всех пользователей тратит между регистрацией и началом обучения менее 3 часов 28 минут.
Такая сегментация пользователей может дать нам важные опорные точки для оптимизации времени. Мы можем сосредоточится не на оптимизации среднего времени (поскольку среднее будет увеличено за счёт пользователей, которые долго проходят этап), а на том, чтобы оптимизировать медианное время.

Также мы можем отдельно выделить тех пользователей, у которых время больше медианного, чтобы исследовать их детально и понять, что мешает им быстрее дойти до этапа обучения.

#### Измеряем время обучения

In [105]:
"""Сначала сформируем датафрейм tutorial_finish_df, который содержит события окончания обучения."""

tutorial_finish_df = total_events_df[total_events_df["event_type"] == "tutorial_finish"]

In [106]:
"""Давайте возьмём время окончания первого обучения для тех обучений, которые были закончены. 
Для этого сформируем список с идентификаторами первых обучений. Сделать мы это можем, взяв 
уникальные значения столбца tutorial_id в уже созданном датафрейме tutorial_start_df_wo_duplicates. 
Запишем эти обучения в список first_tutorial_ids:"""

first_tutorial_ids = tutorial_start_df_wo_duplicates["tutorial_id"].unique()

In [107]:
"""Теперь отфильтруем датафрейм tutorial_finish_df, оставив в нём только события для таких 
обучений, которые были первыми для пользователя. Мы уже выделили идентификаторы этих обучений, 
и нам не составит труда отфильтровать датафрейм по этим идентификаторам. """

tutorial_finish_df = tutorial_finish_df[
    tutorial_finish_df["tutorial_id"].isin(first_tutorial_ids)
]

In [108]:
"""Проверим, что для каждого пользователя содержится по одному событию завершения обучения:"""

tutorial_finish_df['user_id'].value_counts().mean()

1.0

А дальше произведём те же манипуляции, что и для предыдущих событий:
1. Объединим датафреймы двух событий.
2. Посчитаем разницу во времени между событиями.
3. Выведем характеристики разницы.

Получим следующий код:

In [109]:
tutorial_finish_df = tutorial_finish_df[["user_id", "start_time"]].rename(
    columns={"start_time": "tutorial_finish_time"}
)
merged_df_2 = tutorial_start_df_wo_duplicates.merge(
    tutorial_finish_df, on="user_id", how="inner"
)
merged_df_2["timedelta"] = (
    merged_df_2["tutorial_finish_time"] - merged_df_2["tutorial_start_time"]
)
merged_df_2.head()
print(merged_df_2["timedelta"].mean())
print(merged_df_2["timedelta"].describe())

0 days 00:03:53.174160732
count                         9830
mean     0 days 00:03:53.174160732
std      0 days 00:01:44.803217992
min                0 days 00:00:16
25%                0 days 00:02:28
50%                0 days 00:03:42
75%                0 days 00:05:08
max                0 days 00:10:06
Name: timedelta, dtype: object


#### ВЫВОД
Таким образом, 75 % пользователей проходят обучение менее чем за 5 минут 3 секунды. Неплохой результат!

In [110]:
"""Определяем время между registration и level_choice
По такому же алгоритму рассчитаем время между выбором уровня сложности и регистрацией:"""

level_choice_df = total_events_df[total_events_df["event_type"] == "level_choice"]
print(level_choice_df["user_id"].value_counts().mean())
level_choice_df = level_choice_df[["user_id", "start_time"]].rename(
    columns={"start_time": "level_choice_time"}
)
merged_df_3 = registration_df.merge(level_choice_df, on="user_id", how="inner")
merged_df_3["timedelta"] = (
    merged_df_3["level_choice_time"] - merged_df_3["registration_time"]
)
print(merged_df_3["timedelta"].mean())
print(merged_df_3["timedelta"].describe())

1.0
0 days 07:10:19.169863342
count                         8342
mean     0 days 07:10:19.169863342
std      0 days 04:33:51.164488800
min                0 days 00:08:15
25%         0 days 03:53:16.500000
50%                0 days 06:03:28
75%         0 days 09:34:58.500000
max                1 days 18:48:25
Name: timedelta, dtype: object


In [ ]:
# условие регестрации в 2018
mask = (event['start_time']>='2018-01-01') & (event['start_time']<'2019-01-01') & (event['event_type']=='registration')
# список пользователей, зарег. в 2018
registered = event[mask]['user_id'].to_list()
# новая таблица events. Метод isin фильтрация по списку значений
events_df = event[event['user_id'].isin(registered)]
# приведение даты к формату
events_df['start_time'] = pd.to_datetime(events_df['start_time'])
#просмотр результата
events_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 66959 entries, 51405 to 118364
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              66959 non-null  int64         
 1   event_type      66959 non-null  object        
 2   selected_level  8342 non-null   object        
 3   start_time      66959 non-null  datetime64[ns]
 4   tutorial_id     32954 non-null  float64       
 5   user_id         66959 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 3.6+ MB


<ipython-input-45-3c377d7b8a26>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_df['start_time'] = pd.to_datetime(events_df['start_time'])


→ Сколько в среднем для всех пользователей проходит времени между событием выбора уровня сложности тренировки (level_choice) до события выбора набора бесплатных вопросов (pack_choice)?

In [111]:
pack_choice_df = total_events_df[total_events_df["event_type"] == "pack_choice"]
print(pack_choice_df["user_id"].value_counts().mean())
pack_choice_df = pack_choice_df[["user_id", "start_time"]].rename(
    columns={"start_time": "pack_choice_time"}
)
merged_df_4 = level_choice_df.merge(pack_choice_df, on="user_id", how="inner")
merged_df_4["timedelta"] = (
    merged_df_4["pack_choice_time"] - merged_df_4["level_choice_time"]
)
print(merged_df_4["timedelta"].mean())
print(merged_df_4["timedelta"].describe())

1.0
0 days 00:05:17.128464354
count                         5737
mean     0 days 00:05:17.128464354
std      0 days 00:02:43.923107804
min                0 days 00:00:19
25%                0 days 00:03:05
50%                0 days 00:04:57
75%                0 days 00:07:08
max                0 days 00:15:48
Name: timedelta, dtype: object


→ Сколько в среднем для всех пользователей проходит времени между событием выбора бесплатных вопросов (pack_choice) и первой оплатой (purchase)?

In [123]:
purchase_pack_df = total_events_df[total_events_df["event_type"] == "purchase"]
print(purchase_pack_df["user_id"].value_counts().mean())
purchase_pack_df = purchase_pack_df[["user_id", "event_datetime"]].rename(
    columns={"event_datetime": "purchase_time"}
)
merged_df_5 = purchase_pack_df.merge(pack_choice_df, on="user_id", how="inner")
merged_df_5["timedelta"] = (
merged_df_5["purchase_time"] - merged_df_5["pack_choice_time"]
)
print(merged_df_5["timedelta"].mean())
print(merged_df_5["timedelta"].describe())

1.0
3 days 17:46:53.403125
count                         1600
mean        3 days 17:46:53.403125
std      2 days 04:37:20.225124289
min                0 days 00:44:50
25%         1 days 21:24:13.250000
50%         3 days 12:51:25.500000
75%         5 days 09:42:13.750000
max               10 days 18:33:59
Name: timedelta, dtype: object


## 38.8 PRACTICE. Проверка аналитической гипотезы

Определим, существует ли различие в частоте и средней величине оплат между тремя группами пользователей:

1. пользователи, которые прошли обучение хотя бы раз;
2. пользователи, которые начали обучение, но не прошли его ни разу;
3. пользователи, которые не начинали обучение, а сразу же перешли к выбору уровня сложности.

#### Ищем пользователей, прошедших обучение хотя бы раз

In [124]:
"""Определим пользователей, которые прошли обучение хотя бы раз. Запишем идентификаторы этих
пользователей в переменную users_with_finished_tutorial. Это пользователи, у которых есть 
событие tutorial_finish:"""

users_with_finished_tutorial = total_events_df[
    total_events_df["event_type"] == "tutorial_finish"
]["user_id"].unique()
print(len(users_with_finished_tutorial))

10250


#### Ищем пользователей, которые начали, но не закончили обучение

In [125]:
"""Затем найдём пользователей, которые начали обучение, но не прошли его ни разу. Это будут 
пользователи, у которых есть событие tutorial_start, но нет события tutorial_finish. Проще 
всего сначала найти пользователей, у которых есть событие tutorial_start, а после убрать 
из списка таких пользователей, у которых есть событие tutorial_finish. В этом нам поможет 
работа со множествами:"""

users_with_started_tutorial = total_events_df[
    total_events_df["event_type"] == "tutorial_start"
]["user_id"].unique()
set_users_with_started_tutorial = set(users_with_started_tutorial)
set_users_not_finished_but_started_tutorial = (
    set_users_with_started_tutorial.difference(set(users_with_finished_tutorial))
)

print(len(set_users_with_started_tutorial))
print(len(set_users_not_finished_but_started_tutorial))
print(
    len(set_users_with_started_tutorial) - len(set(users_with_finished_tutorial))
    == len(set_users_not_finished_but_started_tutorial)
)

11858
1608
True


#### Ищем пользователей, сразу выбравших уровень сложности

In [126]:
"""последняя группа пользователей — те, кто ни разу не проходил обучение. У таких пользователей 
отсутствует событие tutorial_start. Поэтому мы можем просто взять и убрать из множества всех 
пользователей множество set_users_with_started_tutorial:"""

all_users = total_events_df["user_id"].unique()
set_all_users = set(all_users)
set_users_not_started_tutorial = set_all_users.difference(
    set_users_with_started_tutorial
)
print(len(set_users_not_started_tutorial))
print(
    len(set_all_users) - len(set_users_with_started_tutorial)
    == len(set_users_not_started_tutorial)
)

8068
True


#### Проверяем результат

In [127]:
"""Просуммировав длину всех множеств пользователей, мы должны получить исходное количество
всех пользователей:"""

len(set_users_not_finished_but_started_tutorial) + len(set_users_not_started_tutorial) + len(users_with_finished_tutorial) == len(set_all_users)

True

#### Считаем пользователей, которые завершили обучение и совершили оплату. Считаем средний размер платежа разных групп

In [137]:
"""посчитаем, сколько пользователей, которые завершили обучение, совершили оплату. Сначала 
сделаем датафрейм purchase_df_1, в котором будут данные по оплатам пользователей, которые 
завершили обучение."""

purchase_df_1 = purchase_df[purchase_df["user_id"].isin(users_with_finished_tutorial)]

In [130]:
"""Затем посчитаем, сколько пользователей в этом датафрейме."""

purchase_df_1['user_id'].nunique()

1447

In [131]:
"""Поделив количество пользователей в датафрейме purchase_df_1 на общее количество пользователей, 
которые завершили обучение, мы получим процент оплативших пользователей в этой группе."""

percent_of_purchase_1 = purchase_df_1["user_id"].nunique() / len(
    users_with_finished_tutorial
)
print(
    "Процент пользователей, которые оплатили тренировки (от числа пользователей, завершивших обучение): {:.2%}".format(
        percent_of_purchase_1
    )
)

Процент пользователей, которые оплатили тренировки (от числа пользователей, завершивших обучение): 14.12%


In [132]:
"""Дальше найдём средний размер платежа для этих пользователей."""

purchase_df_1['amount'].mean()

110.98825155494126

In [133]:
"""Давайте сделаем датафрейм purchase_df_2, в котором будут оплаты пользователей, начавших, 
кончивших обучение. И найдём, какой процент таких пользователей оплачивает пакеты вопросов, 
от общего числа пользователей:"""

purchase_df_2 = purchase_df[
    purchase_df["user_id"].isin(set_users_not_finished_but_started_tutorial)
]
print(purchase_df_2["user_id"].nunique())
percent_of_purchase_2 = purchase_df_2["user_id"].nunique() / len(
    set_users_not_finished_but_started_tutorial
)
print(
    "Процент пользователей, которые оплатили тренировки (от числа пользователей, начавших обучение, но не завершивших): {:.2%}".format(
        percent_of_purchase_2
    )
)

131
Процент пользователей, которые оплатили тренировки (от числа пользователей, начавших обучение, но не завершивших): 8.15%


In [134]:
"""А что у нас со средним чеком?"""

purchase_df_2['amount'].mean()

104.9618320610687

In [135]:
"""Какой процент пользователей, ни разу не начинавших обучение, совершает оплаты?"""

purchase_df_3 = purchase_df[
    purchase_df["user_id"].isin(set_users_not_started_tutorial)
]
print(purchase_df_3["user_id"].nunique())
percent_of_purchase_3 = purchase_df_3["user_id"].nunique() / len(
    set_users_not_started_tutorial
)
print(
    "Процент пользователей, которые оплатили тренировки (от числа пользователей, не начаинавших обучение): {:.2%}".format(
        percent_of_purchase_3
    )
)

22
Процент пользователей, которые оплатили тренировки (от числа пользователей, не начаинавших обучение): 0.27%


In [136]:
"""Средний чек таких пользователей"""

purchase_df_3['amount'].mean()

128.4090909090909

#### DSDJL
Как мы видим, процент пользователей, которые завершили обучение и совершили оплату, выше, чем процент пользователей, которые не начали обучение или не закончили его.

Это говорит о том, что факт прохождения обучения влияет на дальнейшую мотивацию использовать приложение. Существенной разницы в среднем чеке при этом не наблюдается, то есть обучение увеличивает вовлечённость, но выбор количества платных пакетов зависит индивидуально от пользователя.

## 38.9 PROJECT. Исследование поведения пользователей

в файле: gd3.ipynb